In [2]:
# Importing relevant packages 
#!pip3 install python-Levenshtein --user
#!pip3 install fuzzywuzzy --user
import fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
df1 = pd.read_stata # Datensätze einlesen
df1 = pd.read_csv 
df1.head() # Die ersten 5 Einträge des Datensatzes anschauen 
# Sonderzeichen entfernen, alles klein schreiben
df1['hqcompany'] = df1['hqcompany'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

In [ ]:
# Für Unternehmen: Cleaning und Vereinheitlichen von Unternehmenszusätzen wie Corp, Inc 

abbreviations = {
    r'\bincorporated\b': 'inc',
    r'\bcorporation\b': 'corp',
    r'\bltd\b': 'ltd',
    r'\bcompany\b': 'co',
    r'\bthe\b': '',
    r'\b-old\b': '',
    r'\bsystems\b': 'sys',
    r'\blaboratories\b': 'lab',
}

# Function to standardize company names
def standardize_names(name, abbreviations):
    for pattern, replacement in abbreviations.items():
        name = re.sub(pattern, replacement, name, flags=re.IGNORECASE)
    return name

# Apply the standardization to the 'company_name' column
df1['xxx'] = df1['xxx'].apply(lambda x: standardize_names(x, abbreviations))

In [7]:
 df2_subset = df2[df2['xxx'] == 19600]
    # Neuer Datensatz nur auf gewünschter geographischer Einheit

In [ ]:
# Merging data sets 
matched1 = pd.merge(df1_subset, df2_subset, on=['xxx1', 'xxx2'], how='inner')

In [ ]:
# Preparation for Fuzzy Matching, throw out all directly merged variables 
df1_filtered = df1_subset.merge(matched, on=['hqcompany'], how='left', indicator=True)
df1_filtered = df1_filtered[df1_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
# You probably need to throw out and rename some variables after the merge 
df1_filtered = df1_filtered.rename(columns={'dunsnumber_x': 'dunsnumber'})
df1_filtered = df1_filtered.rename(columns={'hqtradename_x': 'hqtradename'})
df1_filtered = df1_filtered.rename(columns={'hqduns_x': 'hqduns'})

In [ ]:
# Function for carrying out the fuzzy match, generates an additional column matched name containing the name from data set 2
def match_names(row, df2_filtered, threshold=90):
    name = row['hqcompany']
    matches = process.extractOne(name, df2_filtered['hqcompany'], scorer=fuzz.token_sort_ratio)
    if matches[1] >= threshold:
        return matches[0]
    return None

df1_filtered['matched_name'] = df1_filtered.apply(match_names, axis=1, df2_filtered=df2_filtered)



In [ ]:
# Matching with the second data set based on the column matched name 
df1_filtered = df1_filtered.rename(columns={'matched_name': 'hqcompany'}) 
matched2 = pd.merge(df2_filtered, df1_filtered, on=['hqcompany', 'CZ_depagri_1990'], how='inner')
# Export Data Set to Stata data set 
df1_cleaned.to_stata('H:/InnovationProject/data/temp/matching_NETS/nonmatch_NETS_19600.dta')